# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d3d66004-72c4-4dd8-a9ee-11bb07bf4dda;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 202ms :: artifacts dl 5m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 4

In [5]:
task4 = (
    parsed_df
        .withWatermark("timestamp", "10 minutes")
        .groupBy(window("timestamp", "5 minutes"), "name")
        .agg(
            sum("n_shares").alias("tot_shares_exchanged"),       
            avg("price").alias("average_price")     
        )
        .withColumn("total_volume", col("tot_shares_exchanged") * col("average_price")) 
        .orderBy(desc("total_volume"))
        .limit(5)   
)

task4_query = (
    task4.writeStream
         .format("console")
         .outputMode("complete")
         .option("truncate", False)
         .start()
)

25/11/24 21:53:54 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7375c133-9547-473b-afa8-ff00ad5e39ff. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+-------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume       |
+------------------------------------------+----+--------------------+------------------+-------------------+
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|CTAS|1396668             |47.369998931884766|6.616016166819763E7|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|OKE |956470              |44.09989929199219 |4.218023067581177E7|
+------------------------------------------+----+--------------------+------------------+-------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+--------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume        |
+------------------------------------------+----+--------------------+------------------+--------------------+
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|BAC |127949706           |14.850000381469727|1.9000531829089394E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|ISRG|14020350            |131.05650329589844|1.8374580459846497E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|JPM |30777400            |53.470001220703125|1.6456676155700684E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|SLB |17436565            |83.55000305175781 |1.4568250589621735E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|BA  |12268943            |108.06566619873047|1.3258514988492508E9|
+--------------

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+--------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume        |
+------------------------------------------+----+--------------------+------------------+--------------------+
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|JPM |61960709            |53.720001220703125|3.328529363115631E9 |
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|BAC |203503069           |14.475000381469727|2.94570700140526E9  |
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|FB  |58428451            |49.869998931884766|2.9138267889616814E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|ISRG|14020350            |131.05650329589844|1.8374580459846497E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|BA  |15018622            |110.94649887084961|1.666263528764717E9 |
+--------------

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+--------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume        |
+------------------------------------------+----+--------------------+------------------+--------------------+
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|JPM |78227924            |55.13333384195963 |4.312966249655446E9 |
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|BAC |203503069           |14.475000381469727|2.94570700140526E9  |
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|FB  |58428451            |49.869998931884766|2.9138267889616814E9|
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|C   |48582342            |50.94999885559082 |2.475270269301922E9 |
|[2025-11-24 21:50:00, 2025-11-24 21:55:00]|SLB |24691575            |88.125            |2.175945046875E9    |
+--------------

...

-------------------------------------------
Batch: 588
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+---------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume         |
+------------------------------------------+----+--------------------+------------------+---------------------+
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|AAPL|2478051899          |113.03713607788086|2.801118897153141E11 |
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AAPL|2491259213          |109.24692366844   |2.721624050809089E11 |
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|FB  |1273148727          |100.87452543349494|1.2842827364238321E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|FB  |1033937902          |114.71167296332283|1.1860474647860812E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AMZN|152004575           |653.6261759440105 |9.935416908324454E10 |
+----

-------------------------------------------
Batch: 589
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+---------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume         |
+------------------------------------------+----+--------------------+------------------+---------------------+
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|AAPL|2478051899          |113.03713607788086|2.801118897153141E11 |
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AAPL|2491259213          |109.24692366844   |2.721624050809089E11 |
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|FB  |1273148727          |100.87452543349494|1.2842827364238321E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|FB  |1033937902          |114.71167296332283|1.1860474647860812E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AMZN|152004575           |653.6261759440105 |9.935416908324454E10 |
+----

-------------------------------------------
Batch: 590
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+---------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume         |
+------------------------------------------+----+--------------------+------------------+---------------------+
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|AAPL|2478051899          |113.03713607788086|2.801118897153141E11 |
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AAPL|2491259213          |109.24692366844   |2.721624050809089E11 |
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|FB  |1273148727          |100.87452543349494|1.2842827364238321E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|FB  |1033937902          |114.71167296332283|1.1860474647860812E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AMZN|152004575           |653.6261759440105 |9.935416908324454E10 |
+----

-------------------------------------------
Batch: 591
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+---------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume         |
+------------------------------------------+----+--------------------+------------------+---------------------+
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|AAPL|2478051899          |113.03713607788086|2.801118897153141E11 |
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AAPL|2491259213          |109.24692366844   |2.721624050809089E11 |
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|FB  |1273148727          |100.87452543349494|1.2842827364238321E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|FB  |1033937902          |114.71167296332283|1.1860474647860812E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AMZN|152004575           |653.6261759440105 |9.935416908324454E10 |
+----

-------------------------------------------
Batch: 592
-------------------------------------------
+------------------------------------------+----+--------------------+------------------+---------------------+
|window                                    |name|tot_shares_exchanged|average_price     |total_volume         |
+------------------------------------------+----+--------------------+------------------+---------------------+
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|AAPL|2478051899          |113.03713607788086|2.801118897153141E11 |
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AAPL|2491259213          |109.24692366844   |2.721624050809089E11 |
|[2025-11-24 21:55:00, 2025-11-24 22:00:00]|FB  |1273148727          |100.87452543349494|1.2842827364238321E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|FB  |1033937902          |114.71167296332283|1.1860474647860812E11|
|[2025-11-24 22:00:00, 2025-11-24 22:05:00]|AMZN|152004575           |653.6261759440105 |9.935416908324454E10 |
+----